### just a single directory preprocessing    
merge $\rightarrow$ demean $\rightarrow$ detrend $\rightarrow$ taper

In [ ]:
# merge->demean->detrend->taper
import obspy
from obspy import read
import os
from obspy.io.sac import attach_paz
from obspy.signal.invsim import corn_freq_2_paz
from obspy.signal import PPSD
import glob
# Define the directory containing your seismic data files (EPZ).
data_directory = '/raid1/SM_data/archive/2023/TW/SM01/EPZ.D/'
output_dir = '/raid1/SM_data/archive/2023/TW/SM01_ppreprocessing'
os.makedirs(output_dir, exist_ok=True)
# exist_ok, when dir is exist that won't exist error.

# Use glob to find all files with a specific extension (e.g., .mseed) in the directory.
data_files = glob.glob(os.path.join(data_directory, '*EPZ*'))

# Loop through each data file and process them.
for data_file in data_files:
    # Read the seismic data file into a Stream object.
    st = read(data_file)

    # Merge traces within the Stream if there is more than one trace.
    if len(st) > 1:
        st = st.merge(method=1, fill_value=0)
        # method 1: more flexible, but for our purpose there's no difference between using method 0 or 1.
        # filled = 0, that's a default, we don't want to interpolate the value.
    # Preprocess the Stream.
    st.detrend('demean')
    st.detrend('linear')
    st.taper(type='hann', max_percentage=0.05)

    # Define the output file path and save the preprocessed Stream.
    output_file = os.path.join(output_dir, os.path.basename(data_file))
    st.write(output_file, format='SAC')

print("Preprocessing and saving completed.")

## instrument response function, method and parameter   
### attach_paz()    
Examples from [here!](<https://docs.obspy.org/packages/autogen/obspy.io.sac.sacpz.attach_paz.html#obspy.io.sac.sacpz.attach_paz>)

In [3]:
from obspy import Trace
from obspy.io.sac import attach_paz
import io
tr = Trace()
f = io.StringIO("""ZEROS 3 
-5.032 0.0
POLES 6
-0.02365 0.02365
-0.02365 -0.02365
-39.3011 0.
-7.74904 0.
-53.5979 21.7494
-53.5979 -21.7494
CONSTANT 2.16e18""")
# using triple quote to create multi-line string
attach_paz(tr, f,torad=True) 
# actually I don't understand the torad, it's not important enough to know.
# the importance of using attach_paz just to attach the AttribDict onto the stats.
tr.stats
# using .stats to confirm 

         network: 
         station: 
        location: 
         channel: 
       starttime: 1970-01-01T00:00:00.000000Z
         endtime: 1970-01-01T00:00:00.000000Z
   sampling_rate: 1.0
           delta: 1.0
            npts: 0
           calib: 1.0
             paz: AttribDict({'seismometer_gain': 1.0, 'digitizer_gain': 1.0, 'poles': [(-0.14859733251479723+0.14859733251479723j), (-0.14859733251479723-0.14859733251479723j), (-246.9360940759956+0j), (-48.6886542727469+0j), (-336.76553777568074+136.6555105199717j), (-336.76553777568074-136.6555105199717j)], 'zeros': [(-31.61698846572768+0j), 0j, 0j], 'sensitivity': 1.0, 'gain': 2.16e+18})

### corn_frq_2_paz() 

### simulate()  